# Análise de Componentes Principais

Atualmente, os dados disponíveis utilizados normalmente contêm diversas *features*, ou seja, diversos preditores que são utilizados por algoritmos de aprendizado de máquina para predição.

Porém, nesse contexto, incluir preditores irrelevantes pode causar efeitos negativos no modelo final, como, por exemplo, aumento do tempo computacional demandado na fase de treinamento, redução de performance, etc.

Ademais, com a grande quantidade de dados utilizados em modelos atualmente, nem sempre é possível analisar e compreender quais preditores são relevantes para a variável resposta, sendo que alguns preditores individuais podem ter alta correlação/associação com a mesma, ou mesmo combinação de preditores. Assim, é importante tentar indentificar uma combinação que seja relevantes para a resposta.

Nesse escopo, temos a análise de Componentes Principais (*Principal Component Analysis - PCA*) como uma técnica de transformação amplamente utilizada em aprendizado de máquina visando redução de dimensionalidade. Especificamente, o objetivo do PCA é encontrar combinações lineares dos preditores originais de modo que as combinações sumarizem o máximo possível de variação (i.e. estatisticamente, perda mínima de informação) no espaço original dos preditores (i.e. subespaço que contém informação relevante).

O método, linear, utiliza uma matriz de preditores numéricos (X) e cria novos componentes (*scores*) que são combinações lineares dos dados originais (por isso o método é linear), sendo que a matriz A é chamada de matriz de projeção:

$$X_{(nXp)}^*=X_{(nxp)}A_{(pxp)}$$

Como resultado, o valor do primeiro score para os dados *i* seriam:

$$x_{i1}^*=a_{a11}x_{i1} + a_{21}x_{i2}+...+a_{p1}x_{ip}$$

Simultaneamente, os novos scores do PCA devem ser ortogonais, ou seja, não-correlacionados entre si. Essa propriedade faz com que a variabilidade do espaço original seja particionada de forma que não se sobreponha e, assim, a variabilidade condensada sobre todos os componentes principais é exatamente igual à variabilidade sobre os preditores originais (i.e. igual variabilidade). Usualmente, seleciona-se um novo set de variáveis que contabilizam um certo montante pré-definido de variabilidade do espaço original (e.g. projeção com $n$ componentes $\to$ 85% da variabilidade original).

Outro benefício dessa ténica é que os scores são não-correlacionados entre si. Esse fator é de grande importância para determinados algoritmos de aprendizado de máquina que possuem performance inferior na presença de colinearidade (e.g. regressão linear múltipla, redes neurais, SVM, etc.).

É importante ter a noção de que a redução de dimensionalidade não deve ser confundida com seleção de preditores, pois ainda que a matriz de projeção tenha menos colunas de preditores, as novas colunas são funções (combinações) de todos os preditores originais.

A ideia seria basicamente:

1 - Calcular a média das colunas da matriz e centralizar, subtraindo a média

2 - Calcular a matriz de covariância

3 - Calcular [autovetores e autovalores](https://pt.wikipedia.org/wiki/Autovalores_e_autovetores) e ordená-los

4 - Selecionar um subset da matriz de autovetores i.e. número de componentes

5 - Transformar os dados através de multiplicação de matrizes: subset transposto do passo 4 e matriz centralizada de médias transposta.

Abaixo vemos o passo a passo da realização da **Anális de Componentes Principais** na mão, e uma comparação dos resultados com o método implementado na biblioteca `sklearn`.

Referências: [[1]](https://www.amazon.com.br/Feature-Engineering-Selection-Practical-Predictive/dp/1138079227)
, [[2]](https://machinelearningmastery.com/calculate-principal-component-analysis-scratch-python/), [[3]](https://www.askpython.com/python/examples/principal-component-analysis).

### PCA from scratch

In [2]:
import numpy as np

1 - Centralizar a matriz:

In [6]:
# criar matriz nxp = 20x10
X = np.random.randint(10,50,200).reshape(20,10)
X

array([[43, 14, 46, 40, 49, 23, 10, 49, 29, 48],
       [13, 16, 24, 32, 39, 27, 17, 28, 38, 44],
       [27, 46, 23, 23, 23, 48, 33, 27, 17, 28],
       [24, 49, 37, 44, 41, 45, 21, 28, 44, 42],
       [33, 38, 17, 20, 17, 36, 23, 39, 20, 24],
       [49, 29, 48, 10, 33, 28, 16, 25, 32, 29],
       [32, 24, 39, 31, 10, 23, 14, 44, 35, 48],
       [42, 22, 28, 37, 17, 18, 39, 31, 47, 10],
       [13, 43, 21, 45, 14, 11, 29, 28, 17, 37],
       [18, 46, 41, 42, 10, 13, 49, 37, 43, 42],
       [15, 23, 13, 18, 36, 35, 42, 37, 14, 17],
       [29, 13, 23, 46, 15, 37, 36, 44, 22, 39],
       [16, 14, 33, 38, 11, 47, 47, 34, 20, 25],
       [18, 47, 45, 37, 21, 49, 10, 27, 43, 39],
       [28, 16, 43, 15, 27, 35, 39, 22, 13, 34],
       [22, 13, 26, 33, 35, 45, 39, 44, 45, 16],
       [40, 44, 43, 31, 37, 47, 44, 12, 30, 17],
       [44, 14, 12, 19, 31, 41, 36, 47, 49, 37],
       [40, 23, 15, 42, 40, 14, 31, 12, 29, 44],
       [20, 38, 48, 36, 26, 18, 31, 13, 28, 15]])

In [7]:
# centralizar colunas tirando média
X_mean = X - np.mean(X, axis=0) 
X_mean

array([[ 14.7 , -14.6 ,  14.75,   8.05,  22.4 ,  -9.  , -20.3 ,  17.6 ,
         -1.75,  16.25],
       [-15.3 , -12.6 ,  -7.25,   0.05,  12.4 ,  -5.  , -13.3 ,  -3.4 ,
          7.25,  12.25],
       [ -1.3 ,  17.4 ,  -8.25,  -8.95,  -3.6 ,  16.  ,   2.7 ,  -4.4 ,
        -13.75,  -3.75],
       [ -4.3 ,  20.4 ,   5.75,  12.05,  14.4 ,  13.  ,  -9.3 ,  -3.4 ,
         13.25,  10.25],
       [  4.7 ,   9.4 , -14.25, -11.95,  -9.6 ,   4.  ,  -7.3 ,   7.6 ,
        -10.75,  -7.75],
       [ 20.7 ,   0.4 ,  16.75, -21.95,   6.4 ,  -4.  , -14.3 ,  -6.4 ,
          1.25,  -2.75],
       [  3.7 ,  -4.6 ,   7.75,  -0.95, -16.6 ,  -9.  , -16.3 ,  12.6 ,
          4.25,  16.25],
       [ 13.7 ,  -6.6 ,  -3.25,   5.05,  -9.6 , -14.  ,   8.7 ,  -0.4 ,
         16.25, -21.75],
       [-15.3 ,  14.4 , -10.25,  13.05, -12.6 , -21.  ,  -1.3 ,  -3.4 ,
        -13.75,   5.25],
       [-10.3 ,  17.4 ,   9.75,  10.05, -16.6 , -19.  ,  18.7 ,   5.6 ,
         12.25,  10.25],
       [-13.3 ,  -5.6 , -18.25

2 - Calcular matriz de variância e covariância:

A matriz de covariância é uma matriz quadrada que contém a covariância entre os elementos, e na diagonal, a sua variância i.e. covariância entre o elemento e ele mesmo.

In [9]:
# matriz de variância e covariância
COV = np.cov(X_mean.T)
COV

array([[ 1.32957895e+02, -3.28210526e+01,  1.28157895e+01,
        -3.86157895e+01,  3.98105263e+01, -8.47368421e+00,
        -2.39894737e+01,  4.45263158e+00,  2.84473684e+01,
        -2.36842105e-01],
       [-3.28210526e+01,  1.86989474e+02,  4.38421053e+01,
         2.11368421e+01, -2.83263158e+01,  5.36842105e+00,
        -1.66105263e+01, -6.91473684e+01,  3.84210526e+00,
        -5.78947368e+00],
       [ 1.28157895e+01,  4.38421053e+01,  1.54618421e+02,
         8.48684211e+00,  3.00000000e+00, -4.21052632e+00,
        -4.08157895e+01, -3.61052632e+01,  2.06447368e+01,
         1.05394737e+01],
       [-3.86157895e+01,  2.11368421e+01,  8.48684211e+00,
         1.17944737e+02, -1.85473684e+01, -3.93157895e+01,
         1.64736842e+00, -3.15789474e-02,  2.89868421e+01,
         3.88289474e+01],
       [ 3.98105263e+01, -2.83263158e+01,  3.00000000e+00,
        -1.85473684e+01,  1.41410526e+02,  2.00000000e+01,
        -3.90315789e+01, -2.31473684e+01,  1.85263158e+01,
         1.

3 - Calcular autovalores e autovetores:

Os autovetores da matriz de covariância são ortogonais entre si, e cada vetor representa um eixo principal. Os maiores autovalores correspondem às maiores variabilidades e, portanto, cada eixo principal com maior autovalor será um eixo capturando a maior variabilidade dos dados e, por isso, ordená-los de maneira decrescente.

In [10]:
# autovalores e autovetores
autovalores, autovetores = np.linalg.eig(COV)

In [11]:
autovalores

array([285.57706705, 275.52198606, 225.24381955, 163.34965725,
       144.16080363, 121.49022412, 102.54340261,  63.86860577,
        45.47843835,  39.45020614])

In [12]:
autovetores

array([[ 0.13721635, -0.30927294, -0.36103493, -0.34494554,  0.23923022,
         0.2245303 ,  0.38528447,  0.5492624 , -0.27985232,  0.00962778],
       [ 0.19501726,  0.68983015, -0.128715  ,  0.20074701, -0.02384926,
        -0.1047248 ,  0.51449175,  0.06509599, -0.02967168, -0.38455167],
       [ 0.37400035,  0.27075408, -0.28182748, -0.02595925,  0.18806582,
         0.52339429, -0.59575157,  0.01238445,  0.05775293, -0.19747483],
       [ 0.21606306,  0.15882632,  0.42082098,  0.07498363,  0.24331578,
        -0.36180072, -0.30811146,  0.31037154, -0.60005446,  0.06282729],
       [ 0.17185216, -0.25623599, -0.45615749, -0.15213615, -0.13198953,
        -0.62660378, -0.23194982, -0.09786934,  0.00106457, -0.4473825 ],
       [-0.26677857, -0.04765958, -0.45088162,  0.74547559, -0.05903555,
        -0.04221416, -0.13549906,  0.30468777, -0.0809473 ,  0.2109866 ],
       [-0.56688152,  0.13885152,  0.13650558, -0.15417068,  0.35611533,
        -0.10591427, -0.17065862,  0.42462374

In [13]:
# ordenar autovalores e autovetores de forma decrescente
sorted_ind = np.argsort(autovalores)[::-1]

autovalores_sort = autovalores[sorted_ind]
autovetores_sort = autovetores[sorted_ind]

In [14]:
autovalores_sort

array([285.57706705, 275.52198606, 225.24381955, 163.34965725,
       144.16080363, 121.49022412, 102.54340261,  63.86860577,
        45.47843835,  39.45020614])

In [16]:
autovetores_sort

array([[ 0.13721635, -0.30927294, -0.36103493, -0.34494554,  0.23923022,
         0.2245303 ,  0.38528447,  0.5492624 , -0.27985232,  0.00962778],
       [ 0.19501726,  0.68983015, -0.128715  ,  0.20074701, -0.02384926,
        -0.1047248 ,  0.51449175,  0.06509599, -0.02967168, -0.38455167],
       [ 0.37400035,  0.27075408, -0.28182748, -0.02595925,  0.18806582,
         0.52339429, -0.59575157,  0.01238445,  0.05775293, -0.19747483],
       [ 0.21606306,  0.15882632,  0.42082098,  0.07498363,  0.24331578,
        -0.36180072, -0.30811146,  0.31037154, -0.60005446,  0.06282729],
       [ 0.17185216, -0.25623599, -0.45615749, -0.15213615, -0.13198953,
        -0.62660378, -0.23194982, -0.09786934,  0.00106457, -0.4473825 ],
       [-0.26677857, -0.04765958, -0.45088162,  0.74547559, -0.05903555,
        -0.04221416, -0.13549906,  0.30468777, -0.0809473 ,  0.2109866 ],
       [-0.56688152,  0.13885152,  0.13650558, -0.15417068,  0.35611533,
        -0.10591427, -0.17065862,  0.42462374

4 - Selecionar um subset da matriz de autovetores reordenada:

Selecionar o subset i.e. o número de componentes.

In [27]:
# selecionar número de componentes principais
n_components = 2

# selecionar subset
autovetores_subset = autovetores_sort[:,0:n_components]
autovalores_subset = autovalores_sort[:2]

In [19]:
autovetores_subset

array([[ 0.13721635, -0.30927294],
       [ 0.19501726,  0.68983015],
       [ 0.37400035,  0.27075408],
       [ 0.21606306,  0.15882632],
       [ 0.17185216, -0.25623599],
       [-0.26677857, -0.04765958],
       [-0.56688152,  0.13885152],
       [-0.02013934, -0.44081417],
       [ 0.29524386, -0.13140056],
       [ 0.4969379 , -0.17652122]])

In [28]:
# não utilzado na projeção, somente para fins de comparação com os resultados do sklearn
autovalores_subset

array([285.57706705, 275.52198606])

6 - Transformar os dados:

Projeção do número de componentes principais selecionados.

In [20]:
X_reduced = np.dot(autovetores_subset.T, X_mean.T).T

In [21]:
X_reduced

array([[ 31.38794333, -27.87194182],
       [ 12.04353787, -12.31703407],
       [-14.05656029,  13.69297277],
       [ 21.49499989,  11.22077826],
       [-11.19023858,  -0.03938066],
       [ 13.84504845,  -5.36958004],
       [ 20.16856552,  -8.93207632],
       [ -8.38101348,  -2.65228529],
       [  2.48671086,  20.39059138],
       [  8.01084183,  21.29262691],
       [-30.96204836,  -5.49755404],
       [ -8.78897701, -13.1388421 ],
       [-25.30338924,   2.22642124],
       [ 21.72684968,  17.25949707],
       [-11.36276955,  -1.06844825],
       [-16.47310292, -16.27807802],
       [ -8.34331523,  19.81191748],
       [ -7.88361009, -33.16234069],
       [  9.27679455,  -6.14422442],
       [  2.30373275,  26.57698061]])

### PCA da biblioteca `sklearn`:

In [22]:
from sklearn.decomposition import PCA

In [23]:
# instanciar PCA com 2 componentes
pca = PCA(n_components=2) 

In [24]:
# fit na matriz
pca.fit(X) 

PCA(n_components=2)

In [25]:
# autovalores
pca.explained_variance_ 

array([285.57706705, 275.52198606])

In [26]:
# autovalores
pca.components_ 

array([[ 0.13721635,  0.19501726,  0.37400035,  0.21606306,  0.17185216,
        -0.26677857, -0.56688152, -0.02013934,  0.29524386,  0.4969379 ],
       [ 0.30927294, -0.68983015, -0.27075408, -0.15882632,  0.25623599,
         0.04765958, -0.13885152,  0.44081417,  0.13140056,  0.17652122]])

In [30]:
# projeção dos dois primeiros componentes principais
pca.transform(X)

array([[ 31.38794333,  27.87194182],
       [ 12.04353787,  12.31703407],
       [-14.05656029, -13.69297277],
       [ 21.49499989, -11.22077826],
       [-11.19023858,   0.03938066],
       [ 13.84504845,   5.36958004],
       [ 20.16856552,   8.93207632],
       [ -8.38101348,   2.65228529],
       [  2.48671086, -20.39059138],
       [  8.01084183, -21.29262691],
       [-30.96204836,   5.49755404],
       [ -8.78897701,  13.1388421 ],
       [-25.30338924,  -2.22642124],
       [ 21.72684968, -17.25949707],
       [-11.36276955,   1.06844825],
       [-16.47310292,  16.27807802],
       [ -8.34331523, -19.81191748],
       [ -7.88361009,  33.16234069],
       [  9.27679455,   6.14422442],
       [  2.30373275, -26.57698061]])